# Идентификация интернет-пользователей. Отчёт

## Содержание
1. [Цели и задачи проекта](#Цели-и-задачи-проекта)
2. [Постановка задачи](#Постановка-задачи)
2. описание исходных и обработанных данных и признаков.
2. первичный анализ признаков
3. первичный визуальный анализ признаков
4. если пришли к каким-то "инсайтам", обнаружили закономерности – изложите
5. описание предобработки данных
6. если создаются признаки, то описание этого процесса
7. кросс-валидация, построение кривых валидации и обучения
8. оценка модели с описанием выбранной метрики. Опишите простыми словами
9. качество модели, как она может применяться
10. выводы: как построенная модель может применяться на практике, какие бизнес-задачи решать. 
11. Подведите итог, опишите, что еще можно сделать, а что сделать не удалось.

## Цели и задачи проекта

Постулируем 2 основные цели проекта:
- учебная
- исследовательская

К учебной части отнесём закрепление теоретических знаний и отработка их на парктике, а именно.

К исследовательской части отнесём следующие задачи:
- постановка ML-задачи
- предобработка входных данных
- exploratory data analysis (визулизация)
- feature engineering
- подбор модели и её гиперпараметров
- проверка модели на кроссвалидации и отложенных выборках
- оценка применения модели к бизнес-задачам

## Описание проекта

В проекте ставится задача по последовательности из нескольких веб-сайтов, посещенных подряд один и тем же человеком идентифицировать этого человека. Идея в том что, пользователи Интернета по-разному переходят по ссылкам, и это может помогать их идентифицировать (кто-то сначала в почту, потом про футбол почитать, затем новости, контакт, потом наконец – работать, кто-то – сразу работать).

## Описание входных данных

Входные данные взяты из [статьи](http://ceur-ws.org/Vol-1703/paper12.pdf) "A Tool for Classification of Sequential Data".

Данные пришли с прокси-серверов Университета Блеза Паскаля и имеют следующий вид. Для каждого пользователя заведен csv-файл с названием user****.csv (где вместо звездочек – 4 цифры, соответствующие ID пользователя), а в нем посещения сайтов записаны в следующем формате: 

_timestamp, site_

Данные устроены следующем образом:

- каталог 3users – пример из 3 файлов для отладки кода предобработки
- в каталог 10users лежат 10 csv-файлов с названием вида "user[USER_ID].csv", где [USER_ID] – ID пользователя
- аналогично для каталога 150users – 150 пользователей

In [5]:
import os
import pandas as pd

In [ ]:
# !unzip capstone_user_identification.zip

In [2]:
PATH_TO_DATA = 'capstone_user_identification'

Посмотрим на данные.

Пользователь 0001 из каталога с 3 пользователями:

In [7]:
pd.read_csv(os.path.join(PATH_TO_DATA, '3users/user0001.csv'))

,timestamp,site
0,2013-11-15 09:28:17,vk.com
1,2013-11-15 09:33:04,oracle.com
2,2013-11-15 09:52:48,oracle.com
3,2013-11-15 11:37:26,geo.mozilla.org
4,2013-11-15 11:40:32,oracle.com
5,2013-11-15 11:40:34,google.com
6,2013-11-15 11:40:35,accounts.google.com
7,2013-11-15 11:40:37,mail.google.com
8,2013-11-15 11:40:40,apis.google.com
9,2013-11-15 11:41:35,plus.google.com


Пользователь 0031 из каталога с 10 пользователями:

In [6]:
pd.read_csv(os.path.join(PATH_TO_DATA, '10users/user0031.csv'))

,timestamp,site
0,2013-11-15 08:12:07,fpdownload2.macromedia.com
1,2013-11-15 08:12:17,laposte.net
2,2013-11-15 08:12:17,www.laposte.net
3,2013-11-15 08:12:17,www.google.com
4,2013-11-15 08:12:18,www.laposte.net
...,...,...
7595,2014-03-28 13:09:06,webmail.laposte.net
7596,2014-03-28 13:09:07,av.mediapostcommunication.net
7597,2014-03-28 13:09:07,www.google.com
7598,2014-03-28 13:09:07,webmail.laposte.net


Пользователь 0006 из каталога с 150 пользователями:

In [9]:
pd.read_csv(os.path.join(PATH_TO_DATA, '150users/user0006.csv'))

,timestamp,site
0,2013-11-15 08:01:09,www.google.fr
1,2013-11-15 08:01:10,www.google.fr
2,2013-11-15 08:01:11,apis.google.com
3,2013-11-15 08:01:12,www.google.com
4,2013-11-15 08:01:16,www.google.fr
...,...,...
21360,2014-05-28 19:10:00,www.facebook.com
21361,2014-05-28 19:10:01,i3.mangareader.net
21362,2014-05-28 19:10:14,www.mangareader.net
21363,2014-05-28 19:10:15,www.facebook.com


## Постановка задачи

Для достижения целей проекта поставим ML-задачу: 
- классификация
- объект: сессия
- целевой признак: id пользователя

Сессия - последовательность n-посещенных сайтов. Размер сессий определяется:
- количеством сайтом
- временем от первого сайта в сессии
Таким образом требуется преобразовать входные данные в вид:

|site1|timestamp1|site2|timestamp2|...|site_n|timestamp_n|user_id|
|---|---|---|---|---|---|---|---|

То есть для каждого пользователя его история посещения разбивается на окна размера $n$ с шагом $m$ ($n$ и $m$ - параметры)

Возможны варианты классификации:
- многоклассовая (целевой признак user_id)
- бинарная классификация (один из пользователей объявляется искомым; для этого пользователя целевой признак принимает значение True, для остальных - False)